# Steam equilibrating with liquid water

Accident scenario: steam leaks into a rigid, insulated tank that is partially filled with water.

The steam and liquid water are not initially at thermal equilibrium, though they are at the same pressure. The steam is at temperature $T_{s,1}$ = 600 C and pressure $P_1$ = 20 MPa. 
The liquid water is initially at $T_{\text{liq},1}$ = 40 C and pressure $P_1$ = 20 MPa.
The total volume of the tank is $V$ = 10 m$^3$ and the volume of the liquid water initially in the tank is $V_{\text{liq},1} = 1 \; \text{m}^3$.

Eventually, the contents of the tank reach a uniform temperature and pressure, $T_2$ and $P_2$. The tank is well-insulated and rigid.

**Problem:** Determine the final temperature and pressure of the water in the tank.

In [1]:
# load necessary modules

# Numpy adds some useful numerical types and functions
import numpy as np

# Cantera will handle thermodynamic properties
import cantera as ct

# Pint gives us some helpful unit conversion
from pint import UnitRegistry
ureg = UnitRegistry()
Q_ = ureg.Quantity

First, what are the initial states of the steam and liquid?

In [2]:
steam = ct.Water()
temp_steam1 = Q_(600, 'degC')
pres1 = Q_(20, 'MPa')
steam.TP = temp_steam1.to('K').magnitude, pres1.to('Pa').magnitude
steam()


  water:

       temperature   873.15 K
          pressure   2e+07 Pa
           density   55.012 kg/m^3
  mean mol. weight   18.016 kg/kmol
    vapor fraction   1
   phase of matter   supercritical

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.2433e+07         -2.24e+08  J
   internal energy       -1.2797e+07       -2.3055e+08  J
           entropy             10025        1.8061e+05  J/K
    Gibbs function       -2.1186e+07       -3.8169e+08  J
 heat capacity c_p            2808.2             50592  J/K
 heat capacity c_v            1964.4             35391  J/K



In [3]:
liquid = ct.Water()
temp_liquid1 = Q_(40, 'degC')
liquid.TP = temp_liquid1.to('K').magnitude, pres1.to('Pa').magnitude
liquid()


  water:

       temperature   313.15 K
          pressure   2e+07 Pa
           density   1000.8 kg/m^3
  mean mol. weight   18.016 kg/kmol
    vapor fraction   0
   phase of matter   liquid

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.5786e+07       -2.8439e+08  J
   internal energy       -1.5806e+07       -2.8475e+08  J
           entropy            4084.5             73587  J/K
    Gibbs function       -1.7065e+07       -3.0744e+08  J
 heat capacity c_p            4128.7             74382  J/K
 heat capacity c_v            4011.5             72271  J/K



State $s,1$ is in the superheated vapor region (vapor fraction: 1) and state $\text{liq},1$ is in the compressed liquid region (vapor fraction: 0).

We can calculate the mass of liquid water in the tank, and then determine the volume and mass of steam:

$$
m_{\text{liq},1} = \frac{V_{\text{liq},1}}{v_{\text{liq},1}} \\
V_{s,1} = V_{\text{tank}} - V_{\text{liq},1} \\
m_{s,1} = \frac{V_{s,1}}{v_{s,1}}
$$

In [4]:
vol_tank = Q_(10, 'meter^3')
vol_liquid1 = Q_(1, 'meter^3')

mass_liquid1 = vol_liquid1 / Q_(liquid.v, 'm^3/kg')
print(f'Mass of liquid at state 1: {mass_liquid1: .2f}')

vol_steam1 = vol_tank - vol_liquid1
mass_steam1 = vol_steam1 / Q_(steam.v, 'm^3/kg')
print(f'Mass of steam at state 1: {mass_steam1: .2f}')

mass_1 = mass_liquid1 + mass_steam1
print(f'Total mass of system: {mass_1: .2f}')

Mass of liquid at state 1: 1000.81 kilogram
Mass of steam at state 1: 495.11 kilogram
Total mass of system: 1495.92 kilogram


Do a mass balance on the tank for going from state 1 to state 2 (after equilibrium):
\begin{equation}
\frac{dm}{dt} = m_2 - m_1 = 0
\end{equation}
and so $m_2 = m_1$, meaning the mass of water in the tank does not change.
We can then find the specific volume of water in the tank:
\begin{equation}
v_2 = \frac{V_{\text{tank}}}{m_2}
\end{equation}

The tank experiences no heat or work, and does not exhibit any bulk changes in kinetic or potential energy; we can do an energy balance on the tank for the process of going from state 1 to state 2:

$$
Q - W = \Delta KE + \Delta PE + \Delta U \\
\rightarrow 0 = U_2 - U_1 \\
0 = u_2 m_2 - \left( u_{\text{liq},1} m_{\text{liq},1} + u_{s,1} m_{s,1} \right)
$$

Therefore, we can find the specific internal energy of the final state:
\begin{equation}
u_2 = \frac{ u_{\text{liq},1} m_{\text{liq},1} + u_{s,1} m_{s,1} }{m_2}
\end{equation}

In [5]:
mass_2 = mass_1

spec_vol2 = vol_tank / mass_2
print(f'Specific volume of state 2: {spec_vol2: .2e}')

int_energy2 = (
    Q_(liquid.u, 'J/kg')*mass_liquid1 + Q_(steam.u, 'J/kg')*mass_steam1
    ) / mass_2
int_energy2.ito('kilojoule/kg')
print(f'Internal energy of state 2: {int_energy2: .2f}')

water_equilibrium = ct.Water()
water_equilibrium.UV = int_energy2.to('J/kg').magnitude, spec_vol2.to('m^3/kg').magnitude

water_equilibrium()

Specific volume of state 2: 6.68e-03 meter ** 3 / kilogram
Internal energy of state 2: -14809.79 kilojoule / kilogram

  water:

       temperature   510.44 K
          pressure   3.1852e+06 Pa
           density   149.59 kg/m^3
  mean mol. weight   18.016 kg/kmol
    vapor fraction   0.088707
   phase of matter   liquid-gas-mix

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.4789e+07       -2.6643e+08  J
   internal energy        -1.481e+07       -2.6681e+08  J
           entropy              6506        1.1721e+05  J/K
    Gibbs function       -1.8109e+07       -3.2626e+08  J
 heat capacity c_p   inf               inf              J/K
 heat capacity c_v   nan               nan              J/K



So, at equilibrium, the tank contains a mixture of saturated liquid and vapor, with temperature and pressure:

In [6]:
final_temperature = Q_(water_equilibrium.T, 'K')
final_pressure = Q_(water_equilibrium.P, 'Pa')

print(f'Final temperature: {final_temperature: .2f}')
print(f'Final pressure: {final_pressure.to(ureg.MPa): .2f}')

Final temperature: 510.44 kelvin
Final pressure: 3.19 megapascal
